#### KoAlpaca Tensor 병렬 작동

1. ##### requirement.txt
   * deepspeed
   * evaluate
   * accelerate==0.21.0
   * datasets
   * frozenlist<1.4
   * ordereduuid<1
   * pandas<2
   * python-dateutil<2.9
   * requests<3
   * scikit-learn<=1.2
   * scipy<1.10
   * torch>=2.0.0
   * tqdm
   * transformers<4.30
   * typed-argument-parser<1.8
   * wandb
   * urllib3<2
   * pytest
   * tensor_parallel

In [ ]:
import torch
from transformers import pipeline, AutoModelForCausalLM

In [ ]:
model_id = "/data/bwllm/models/KoAlpaca-Polyglot-12.8B"

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    load_in_8bit=True,
    revision="8bit",
    # max_memory=f'{int(torch.cuda.mem_get_info()[0]/1024**3)-2}GB'
)

In [ ]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=model_id,
    # device=2,
)

In [ ]:
def make_answer(state, text):
    
    messages = state + [ {"role": "질문", "content": text}]    
    conversation_history = "\n".join(
        [f"### {msg['role']}:\n{msg['content']}" for msg in messages]
    )
    
    ans = pipe(
        conversation_history + "\n\n### 답변:",
        do_sample=True,
        max_new_tokens=2000,
        temperature=0.7,
        top_p=0.9,
        return_full_text=False,
        eos_token_id=2,
    )
    
    print(ans)
    
    msg = "학습데이터 준비 ###고경호 상무가 감리가이드를 중심으로 데이터를 생성 중에 있습니다."
    if "###" in msg:
        msg = msg.split("###")[0]
        
    return msg
        
        

In [ ]:
state = [
            {
                "role": "맥락",
                "content": "KoAlpaca(코알파카)는 EleutherAI에서 개발한 Polyglot-ko 라는 한국어 모델을 기반으로, 자연어 처리 연구자 Beomi가 개발한 모델입니다.",
            },
            {
                "role": "맥락",
                "content": "ChatKoAlpaca(챗코알파카)는 KoAlpaca를 채팅형으로 만든 것입니다.",
            },
            {"role": "명령어", "content": "친절한 AI 챗봇인 ChatKoAlpaca 로서 답변을 합니다."},
            {
                "role": "명령어",
                "content": "인사에는 짧고 간단한 친절한 인사로 답하고, 아래 대화에 간단하고 짧게 답해주세요.",
            },
        ]

llm_anwser = "대한민국 수도의 이름만 알려줘"
make_answer(state, llm_anwser)